# Visualizing internal tides 

In [1]:
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use('Agg')
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter

import numpy as np

from joblib import Parallel
from joblib import delayed as jb_delayed

In [ ]:
! export  HDF5_USE_FILE_LOCKING=FALSE

## 1. - Data import 

In [ ]:
ds = xr.open_mfdataset("/bettik/bellemva/MITgcm/MITgcm_it/by_mode/MITgcm_it_2012060*.nc",combine='nested',concat_dim="time",parallel=True)
ds = ds.load()

## 2. - Plot 

In [ ]:
def create_plot(date,ssh_it1,ssh_it2,ssh_it3):

    fig, ax = plt.subplots(1,3,figsize=(13,6),subplot_kw={"projection": ccrs.PlateCarree(central_longitude=195.)},dpi=300)

    for i,title,ssh,v_ex in zip(range (3),
                    ["Mode 1","Mode 2","Mode3"],
                    [ssh_it1,ssh_it2,ssh_it3],
                    [10,4,2]):
        ax[i].set_extent([185, 205,15, 35],crs=ccrs.PlateCarree())

        plot = ax[i].pcolormesh(ds.longitude.values, 
                ds.latitude.values, 
                100*ssh,
                vmin=-v_ex,vmax=v_ex,
                cmap="RdBu",
                transform=ccrs.PlateCarree()
                )

        ax[i].add_feature(cfeature.LAND,color="lightgrey",zorder=1)
        #ax[i].add_feature(cfeature.OCEAN,color="black",zorder=0)
        ax[i].coastlines(lw=0.5)

        gridlines = ax[i].gridlines(
            alpha=0.3,
            color="black",
            draw_labels=True,
            xlocs=np.arange(-175,-155,5),
            ylocs=np.arange(20,50,5),
            # xformatter=cartopy.mpl.ticker.LongitudeFormatter(zero_direction_label=True,),
        )
        gridlines.right_labels = False
        gridlines.top_labels  = False
        gridlines.xlabel_style = {'size': 8}
        gridlines.ylabel_style = {'size': 8}
        ax[i].set_aspect("equal")

        cbar = plt.colorbar(plot,ax=ax[i],location="bottom",shrink=0.7)
        cbar.set_label("cm",labelpad=-35)

        ax[i].set_title(title,size=18)

        fig.suptitle("Internal Tide : "+date.astype('datetime64[D]').astype('str'),size=24)

        plt.savefig("/home/bellemva/CCS/data/filter_tool/extract_modes/frames/modes"+date.astype('datetime64[h]').astype("str")+".png")


In [ ]:
Parallel(n_jobs=12,backend='multiprocessing')(jb_delayed(create_plot)(ds.time[i].values,
                                                                      ds.ssh_it1[i].values,
                                                                      ds.ssh_it2[i].values,
                                                                      ds.ssh_it3[i].values) for i in range(ds.dims["time"]))

## 3. - Analysing plots (and anomalys)

In [ ]:
ssh_it1 = ds.ssh_it1.values

In [ ]:
array_diff = np.nansum((ssh_it1[:-1]-ssh_it1[1:])**2,axis=(1,2))

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
ax.bar(ds.time[1:15].values,array_diff[:14])


In [ ]:
ssh = ssh_it1[10]-ssh_it1[9]
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.pcolormesh(ds.longitude.values,ds.latitude.values,ssh,vmin=-0.05,vmax=0.05,cmap="RdBu")
ax.set_aspect("equal")


In [ ]:
ssh = ssh_it1[3]
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.pcolormesh(ds.longitude.values,ds.latitude.values,ssh,vmin=-0.05,vmax=0.05)
ax.set_aspect("equal")


In [ ]:
path_config = '/home/bellemva/CCS/Configs/config_2022a_4DVAR_QG_SWOT.py'

In [ ]:
import sys
sys.path.append('/home/bellemva/MASSH/mapping')
from src import exp
config = exp.Exp(path_config)

In [ ]:
ds.latitude

In [ ]:
ssh_igw = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_20120601.nc").ssh_igw.load()
ssh_igw = ssh_igw.sel(longitude = slice(185.,205.),latitude = slice(15.,35.),drop=True)

In [ ]:

fig,ax = plt.subplots(2,2,figsize=(10,10))
ax[0,0].pcolormesh(ds.longitude.values,ds.latitude.values,ssh_igw[2],vmin=-0.10,vmax=0.10,cmap="RdBu")
ax[0,0].set_title("IGW : 2012-06-01T02")
ax[0,1].pcolormesh(ds.longitude.values,ds.latitude.values,ssh_igw[3],vmin=-0.10,vmax=0.10,cmap="RdBu")
ax[0,1].set_title("IGW : 2012-06-01T03")
ax[1,0].pcolormesh(ds.longitude.values,ds.latitude.values,ssh_it1[2],vmin=-0.10,vmax=0.10,cmap="RdBu")
ax[1,0].set_title("IT : 2012-06-01T02")
ax[1,1].pcolormesh(ds.longitude.values,ds.latitude.values,ssh_it1[3],vmin=-0.10,vmax=0.10,cmap="RdBu")
ax[1,1].set_title("IT : 2012-06-01T03")


## 4. - Plot of zones on the North Pacific

In [6]:
date = np.datetime64("2012-06-15")

ds_hawaii = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/hawaii/MITgcm_it_"+date.astype('str').replace("-","")+".nc")
ds_crossover1 = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/crossover_CCS/MITgcm_it_"+date.astype('str').replace("-","")+".nc")
ds_crossover2 = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/crossover_centerpacific/MITgcm_it_"+date.astype('str').replace("-","")+".nc")
ds_crossover3 = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/crossover_hawaii/MITgcm_it_"+date.astype('str').replace("-","")+".nc")

#for ds in [ds_hawaii,ds_crossover1,ds_crossover2,ds_crossover3]:
    #ds = ds.coarsen(longitude=4,latitude=4,boundary='trim').mean()
    

In [2]:
track_swot = xr.open_dataset("/bettik/bellemva/track_swot_pacific.nc")
track_swot

<xarray.Dataset>
Dimensions:          (num_lines: 21361, num_pixels: 69)
Coordinates:
    latitude         (num_lines, num_pixels) float64 ...
    longitude        (num_lines, num_pixels) float64 ...
    latitude_nadir   (num_lines) float64 ...
    longitude_nadir  (num_lines) float64 ...
Dimensions without coordinates: num_lines, num_pixels
Data variables:
    ssh_karin        (num_lines, num_pixels) float64 ...
Attributes: (12/55)
    Conventions:                               CF-1.7
    title:                                     Level 2 Low Rate Sea Surface H...
    source:                                    Ka-band radar interferometer
    history:                                   2023-01-24T08:58:04Z : Creation
    platform:                                  SWOT
    reference_document:                        D-56407_SWOT_Product_Descripti...
    ...                                        ...
    ellipsoid_flattening:                      0.0033528106647474805
    institution:                               CNES
    references:                                V0.6
    equator_time:                              2023-01-21T00:24:38.155000Z
    equator_longitude:                         151.02
    product_version:                           01

In [3]:
def form(i):
    if i//10==0:
        return "0"+str(i)+':00'
    else : 
        return str(i)+':00'

In [4]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

date = np.datetime64("2012-06-01")
i=12
v_ex = 7

def plot_hour(i,ds_hawaii,ds_crossover1,ds_crossover2,ds_crossover3):

    fig, ax = plt.subplots(1,1,figsize=(16,9),subplot_kw={"projection": ccrs.PlateCarree(central_longitude=200.)},dpi=300)

    ax.set_extent([160, 260,0, 60],crs=ccrs.PlateCarree())

    ax.add_feature(cfeature.OCEAN,color="snow",zorder=0)
    ax.add_feature(cfeature.LAND.with_scale('50m'),color="lightgrey",zorder=3,edgecolor='none')
    ax.coastlines(lw=0.5,zorder=4,resolution="50m")

    ax.scatter(track_swot.longitude,
               track_swot.latitude,
               s=0.001,c="black",alpha=0.06,
               transform=ccrs.PlateCarree())

    plot = ax.pcolormesh(ds_hawaii.longitude.values, 
                ds_hawaii.latitude.values,
                100*ds_hawaii[i,:,:],
                vmin=-v_ex,vmax=v_ex,
                cmap="RdBu",
                alpha=0.8,
                transform=ccrs.PlateCarree()
                )

    ax.pcolormesh(ds_crossover1.longitude.values, 
                ds_crossover1.latitude.values,
                100*ds_crossover1[i,:,:],
                vmin=-v_ex,vmax=v_ex,
                cmap="RdBu",
                alpha=0.8,
                transform=ccrs.PlateCarree()
                )

    ax.pcolormesh(ds_crossover2.longitude.values, 
                ds_crossover2.latitude.values,
                100*ds_crossover2[i,:,:],
                vmin=-v_ex,vmax=v_ex,
                cmap="RdBu",
                alpha=0.8,
                transform=ccrs.PlateCarree()
                )

    ax.pcolormesh(ds_crossover3.longitude.values, 
                ds_crossover3.latitude.values,
                100*ds_crossover3[i,:,:],
                vmin=-v_ex,vmax=v_ex,
                cmap="RdBu",
                alpha=0.8,
                transform=ccrs.PlateCarree()
                )

    ax.plot([180,180,200,200,180],[25,45,45,25,25],transform=ccrs.PlateCarree(),c="black",lw=0.7)
    ax.plot([185,185,205,205,185],[15,35,35,15,15],transform=ccrs.PlateCarree(),c="black",lw=0.7,linestyle="dashed")
    ax.plot([200,200,220,220,200],[25,45,45,25,25],transform=ccrs.PlateCarree(),c="black",lw=0.7)
    ax.plot([225,225,245,245,225],[25,45,45,25,25],transform=ccrs.PlateCarree(),c="black",lw=0.7)

    ax.text(183.5,45.5,"CROSSOVER 3",transform=ccrs.PlateCarree(),fontsize=15)
    ax.text(203.5,45.5,"CROSSOVER 2",transform=ccrs.PlateCarree(),fontsize=15)
    ax.text(228.5,23,"CROSSOVER 1",transform=ccrs.PlateCarree(),fontsize=15)
    ax.text(192,13,"HAWAII",transform=ccrs.PlateCarree(),fontsize=15)

    ax.set_xticks(np.arange(160,280,20), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(0,80,20), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

    ax.text(175,58,date.astype('str')+" "+form(i),transform=ccrs.PlateCarree(),fontsize=15)

    axins1 = inset_axes(
    ax,
    width="1%",  # width: 50% of parent_bbox width
    height="30%",  # height: 5%
    loc="center left",
    )
    axins1.yaxis.set_ticks_position("right")
    axins1.xaxis.set_ticks_position("none")
    fig.colorbar(plot, cax=axins1, orientation="vertical", ticks=[-7,7],extend='both')

    ax.set_aspect("equal")

    fig.savefig("./frames_pacific_mode1/"+date.astype("str").replace("-","")+form(i).split(":")[0])

    return date,i

In [23]:
i=1
ssh_it1_hawaii = ds_hawaii.ssh_it1.load()
ssh_it1_crossover1 = ds_crossover1.ssh_it1.load()
ssh_it1_crossover2 = ds_crossover2.ssh_it1.load()
ssh_it1_crossover3 = ds_crossover3.ssh_it1.load()
plot_hour(i,ssh_it1_hawaii,ssh_it1_crossover1,ssh_it1_crossover2,ssh_it1_crossover3)

(numpy.datetime64('2012-06-01'), 1)

In [6]:
for date in np.arange(np.datetime64("2012-06-01"),np.datetime64("2012-07-01")):
    
    ds_hawaii = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/hawaii/MITgcm_it_"+date.astype('str').replace("-","")+".nc")
    ds_crossover1 = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/crossover_CCS/MITgcm_it_"+date.astype('str').replace("-","")+".nc")
    ds_crossover2 = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/crossover_centerpacific/MITgcm_it_"+date.astype('str').replace("-","")+".nc")
    ds_crossover3 = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/crossover_hawaii/MITgcm_it_"+date.astype('str').replace("-","")+".nc")

    ssh_it1_hawaii = ds_hawaii.ssh_it1.load()
    ssh_it1_crossover1 = ds_crossover1.ssh_it1.load()
    ssh_it1_crossover2 = ds_crossover2.ssh_it1.load()
    ssh_it1_crossover3 = ds_crossover3.ssh_it1.load()

    Parallel(n_jobs=24,backend='multiprocessing')(jb_delayed(plot_hour)(i,ssh_it1_hawaii,ssh_it1_crossover1,ssh_it1_crossover2,ssh_it1_crossover3) for i in range(24))

    del ssh_it1_hawaii, ssh_it1_crossover1, ssh_it1_crossover2, ssh_it1_crossover3
    print(date)

2012-06-01
2012-06-02
2012-06-03
2012-06-04
2012-06-05
2012-06-06
2012-06-07
2012-06-08
2012-06-09
2012-06-10
2012-06-11
2012-06-12
2012-06-13
2012-06-14
2012-06-15
2012-06-16
2012-06-17
2012-06-18
2012-06-19
2012-06-20
2012-06-21
2012-06-22
2012-06-23
2012-06-24
2012-06-25
2012-06-26
2012-06-27
2012-06-28
2012-06-29
2012-06-30


In [ ]:
plot_hour(i,date)